In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stats
from scipy.stats import f, norm, pearsonr, ttest_rel

!pip install --upgrade gensim
import re
from itertools import permutations

import gensim.downloader as api
from gensim.models import KeyedVectors
from scipy.spatial.distance import cosine, euclidean, pdist

# Load data
study_name = "Story Beliefs"
specification_name = "default persona"
human_file = f"{study_name} human data values anonymized.csv"
twin_file = f"{study_name} twins data values anonymized.csv"
df_human = pd.read_csv(human_file, header=0, skiprows=[1, 2])
df_twin = pd.read_csv(twin_file, header=0, skiprows=[1, 2])


#########################
# create new relevant columns:

# define your weights once
weights = np.arange(1, 6)

# for each dataframe...
for df in (df_human, df_twin):
    # loop over A vs B, chap1 vs chap2, val vs aro
    for prefix in ["A", "B"]:
        for chap in [1, 2]:
            for metric in ["val", "aro"]:
                # build the five source columns
                cols = [f"{prefix}chap{chap}_{metric}_{i}" for i in range(1, 6)]

                # new column name, e.g. "Achap1_val" or "Bchap2_aro"
                newcol = f"{prefix}chap{chap}_{metric}"

                # compute denominator (sum of counts)
                denom = df[cols].sum(axis=1)

                # weighted sum divided by total count
                df[newcol] = df[cols].dot(weights) / denom

                # optional: set to NaN where there were no counts
                df.loc[denom == 0, newcol] = np.nan
###############################


human_file = f"{study_name} human data values anonymized processed.csv"
twin_file = f"{study_name} twins data values anonymized processed.csv"
df_twin.to_csv(twin_file, index=False)
df_human.to_csv(human_file, index=False)

df_human = df_human.set_index("TWIN_ID")
df_twin = df_twin.set_index("TWIN_ID")


##################now proceed with standard meta-analysis steps
# define relevant columns:
# condition variable names:
condition_vars = [""]
# Check if we have a real condition var
if condition_vars and condition_vars[0].strip():
    cond = condition_vars[0]
    cond_h = f"{cond}_human"
    cond_t = f"{cond}_twin"
    cond_exists = True
else:
    cond_exists = False

# raw responses:
# all the “raw” columns
# raw_vars = [
#     f"{prefix}chap{chap}_{metric}_{i}"
#     for prefix in ["A", "B"]
#     for chap   in [1, 2]
#     for metric in ["val", "aro"]
#     for i      in range(1, 6)
# ]
# # check
# print(len(raw_vars), "columns:", raw_vars)
# raw_vars_min = [0]*40
# raw_vars_max = [100]*40
# #raw responses: domain=social?
# raw_vars_social=[0]*40
# raw_vars_social_map = dict(zip(raw_vars, raw_vars_social))
# #raw responses: domain=cognitive?
# raw_vars_cognitive=[0]*40
# raw_vars_cognitive_map = dict(zip(raw_vars, raw_vars_cognitive))
# #raw responses: replicating know human bias?
# raw_vars_known=[0]*40
# raw_vars_known_map = dict(zip(raw_vars, raw_vars_known))
# #raw responses: preference measure?
# raw_vars_pref=[0]*40
# raw_vars_pref_map = dict(zip(raw_vars, raw_vars_pref))
# #raw responses: stimuli dependent?
# raw_vars_stim=[1]*40
# raw_vars_stim_map = dict(zip(raw_vars, raw_vars_stim))

# DVs:
DV_vars = [
    f"{prefix}chap{chap}_{metric}"
    for prefix in ["A", "B"]
    for chap in [1, 2]
    for metric in ["val", "aro"]
]
DV_vars_min = [1] * 8
DV_vars_max = [5] * 8
# DVs: domain=social?
DV_vars_social = [0] * 8
DV_vars_social_map = dict(zip(DV_vars, DV_vars_social))
# DVs: domain=cognitive?
DV_vars_cognitive = [0] * 8
DV_vars_cognitive_map = dict(zip(DV_vars, DV_vars_cognitive))
# DVs: replicating know human bias?
DV_vars_known = [0] * 8
DV_vars_known_map = dict(zip(DV_vars, DV_vars_known))
# DVs: preference measure?
DV_vars_pref = [0] * 8
DV_vars_pref_map = dict(zip(DV_vars, DV_vars_pref))
# DVs: stimuli dependent?
DV_vars_stim = [1] * 8
DV_vars_stim_map = dict(zip(DV_vars, DV_vars_stim))
# DVs: knowledge question?
DV_vars_know = [0] * 8
DV_vars_know_map = dict(zip(DV_vars, DV_vars_know))
# DVs: political question?
DV_vars_politics = [0] * 8
DV_vars_politics_map = dict(zip(DV_vars, DV_vars_politics))

# merging key
merge_key = ["TWIN_ID"]

# Merge on TWIN_ID
df = pd.merge(df_human, df_twin, on=merge_key, suffixes=("_human", "_twin"))

# Fix dtypes
# for var in raw_vars + DV_vars:
for var in DV_vars:
    df[f"{var}_human"] = pd.to_numeric(df[f"{var}_human"], errors="coerce")
    df[f"{var}_twin"] = pd.to_numeric(df[f"{var}_twin"], errors="coerce")

# build min/max maps from both raw_vars and DV_vars
min_map = {v: mn for v, mn in zip(DV_vars, DV_vars_min)}
# min_map = {v: mn for v, mn in zip(raw_vars,      raw_vars_min)}
# min_map.update({v: mn for v, mn in zip(DV_vars,   DV_vars_min)})

max_map = {v: mx for v, mx in zip(DV_vars, DV_vars_max)}
# max_map = {v: mx for v, mx in zip(raw_vars,      raw_vars_max)}
# max_map.update({v: mx for v, mx in zip(DV_vars,   DV_vars_max)})

# now add _min and _max columns for every variable in the union
for var in min_map:
    df[f"{var}_min"] = min_map[var]
    df[f"{var}_max"] = max_map[var]

# Compute results
results = []
# for var in raw_vars:
#     col_h = f"{var}_human"
#     col_t = f"{var}_twin"
#     min_col = f"{var}_min"
#     max_col = f"{var}_max"
#     if cond_exists:
#         cols = [col_h, col_t, cond_h, cond_t,min_col,max_col]
#     else:
#         cols = [col_h, col_t,min_col,max_col]
#     pair = (
#     df[cols]
#       .dropna(subset=[col_h, col_t])
#     )
#     min_val = pair[min_col].iloc[0]
#     max_val = pair[max_col].iloc[0]
#     n    = len(pair)
#     if n >= 4:
#         r, _    = pearsonr(pair[col_h], pair[col_t])
#         z_f     = np.arctanh(r)
#         se      = 1 / np.sqrt(n - 3)
#         z_crit  = norm.ppf(0.975)
#         lo_z, hi_z = z_f - z_crit*se, z_f + z_crit*se
#         lo_r, hi_r = np.tanh(lo_z), np.tanh(hi_z)
#         z_score    = z_f / se
#         # Accuracy = mean absolute diff / range
#         if pd.isna(min_val) or pd.isna(max_val) or max_val == min_val:
#             accuracy = np.nan
#         else:
#             # compute mean absolute difference
#             abs_diff      = np.abs(pair[col_h] - pair[col_t])
#             mean_abs_diff = abs_diff.mean()
#             accuracy      = 1 - mean_abs_diff / (max_val - min_val)

#         mean_h = pair[col_h].mean()
#         mean_t = pair[col_t].mean()

#         # Paired t‐test
#         t_stat, p_val = ttest_rel(pair[col_h], pair[col_t])

#         std_h = pair[col_h].std(ddof=1)
#         std_t = pair[col_t].std(ddof=1)

#          # F‐test for equal variances
#         df1 = df2 = n - 1
#         f_stat = (std_h**2 / std_t**2) if std_t>0 else np.nan

#         # two‐tailed p‐value:
#         if not np.isnan(f_stat):
#             p_f = 2 * min(f.cdf(f_stat, df1, df2),
#                           1 - f.cdf(f_stat, df1, df2))
#         else:
#             p_f = np.nan

#         # Effect sizes (Cohen's d) across conditions
#         #    For humans:
#         if cond_exists and len(pair)>3:
#             levels_h = pair[cond_h].unique()
#             if len(levels_h) == 2:
#                 g1 = pair.loc[pair[cond_h]==levels_h[0], col_h]
#                 g2 = pair.loc[pair[cond_h]==levels_h[1], col_h]
#                 n1, n2 = len(g1), len(g2)
#                 # pooled sd
#                 s_pool = np.sqrt(((n1-1)*g1.var(ddof=1)+(n2-1)*g2.var(ddof=1)) / (n1+n2-2))
#                 d_human = (g1.mean() - g2.mean()) / s_pool if s_pool>0 else np.nan
#             else:
#                 d_human = np.nan
#         else:
#             d_human = np.nan

#         #    For twins:
#         if cond_exists and len(pair)>3:
#             levels_t = pair[cond_t].unique()
#             if cond_exists and len(levels_t) == 2:
#                 g1 = pair.loc[pair[cond_t]==levels_t[0], col_t]
#                 g2 = pair.loc[pair[cond_t]==levels_t[1], col_t]
#                 n1, n2 = len(g1), len(g2)
#                 s_pool = np.sqrt(((n1-1)*g1.var(ddof=1)+(n2-1)*g2.var(ddof=1)) / (n1+n2-2))
#                 d_twin = (g1.mean() - g2.mean()) / s_pool if s_pool>0 else np.nan
#             else:
#                 d_twin = np.nan
#         else:
#             d_twin = np.nan
#     else:
#         r = lo_r = hi_r = z_score = accuracy = mean_h = mean_t = t_stat = p_val = std_h = std_t = f_stat = p_f = np.nan
#         d_human = d_twin = np.nan


#     results.append({
#         'study name': study_name,
#         'variable name': var,
#         'variable type (raw response/DV)':     'raw',
#         'correlation between the responses from humans vs. their twins':        r,
#         'CI_lower': lo_r,
#         'CI_upper': hi_r,
#         'z-score for correlation between humans vs. their twins':  z_score,
#         'accuracy between humans vs. their twins': accuracy,
#         'mean_human': mean_h,
#         'mean_twin': mean_t,
#         'paired t-test t-stat': t_stat,
#         'paired t-test p-value': p_val,
#         'std_human': std_h,
#         'std_twin': std_t,
#         'variance test F-stat': f_stat,
#         'variance test p-value': p_f,
#         'effect size based on human': d_human,
#         'effect size based on twin': d_twin,
#         'domain=social?':raw_vars_social_map.get(var, np.nan),
#         'domain=cognitive?':raw_vars_cognitive_map.get(var, np.nan),
#         'replicating know human bias?':raw_vars_known_map.get(var, np.nan),
#         'preference measure?':raw_vars_pref_map.get(var, np.nan),
#         'stimuli dependent?':raw_vars_stim_map.get(var, np.nan),
#         'sample size':        n
#     })

for var in DV_vars:
    col_h = f"{var}_human"
    col_t = f"{var}_twin"
    min_col = f"{var}_min"
    max_col = f"{var}_max"
    if cond_exists:
        cols = [col_h, col_t, cond_h, cond_t, min_col, max_col]
    else:
        cols = [col_h, col_t, min_col, max_col]
    pair = df[cols].dropna(subset=[col_h, col_t])
    min_val = pair[min_col].iloc[0]
    max_val = pair[max_col].iloc[0]
    n = len(pair)
    if n >= 4:
        r, _ = pearsonr(pair[col_h], pair[col_t])
        z_f = np.arctanh(r)
        se = 1 / np.sqrt(n - 3)
        z_crit = norm.ppf(0.975)
        lo_z, hi_z = z_f - z_crit * se, z_f + z_crit * se
        lo_r, hi_r = np.tanh(lo_z), np.tanh(hi_z)
        z_score = z_f / se
        # Accuracy = mean absolute diff / range
        if pd.isna(min_val) or pd.isna(max_val) or max_val == min_val:
            accuracy = np.nan
        else:
            # compute mean absolute difference
            abs_diff = np.abs(pair[col_h] - pair[col_t])
            mean_abs_diff = abs_diff.mean()
            accuracy = 1 - mean_abs_diff / (max_val - min_val)

        mean_h = pair[col_h].mean()
        mean_t = pair[col_t].mean()

        # Paired t‐test
        t_stat, p_val = ttest_rel(pair[col_h], pair[col_t])

        std_h = pair[col_h].std(ddof=1)
        std_t = pair[col_t].std(ddof=1)

        # F‐test for equal variances
        df1 = df2 = n - 1
        f_stat = (std_h**2 / std_t**2) if std_t > 0 else np.nan
        # two‐tailed p‐value:
        if not np.isnan(f_stat):
            p_f = 2 * min(f.cdf(f_stat, df1, df2), 1 - f.cdf(f_stat, df1, df2))
        else:
            p_f = np.nan

        # Effect sizes (Cohen's d) across conditions
        #    For humans:
        if cond_exists and len(pair) > 3:
            levels_h = pair[cond_h].unique()
            if len(levels_h) == 2:
                g1 = pair.loc[pair[cond_h] == levels_h[0], col_h]
                g2 = pair.loc[pair[cond_h] == levels_h[1], col_h]
                n1, n2 = len(g1), len(g2)
                # pooled sd
                s_pool = np.sqrt(
                    ((n1 - 1) * g1.var(ddof=1) + (n2 - 1) * g2.var(ddof=1)) / (n1 + n2 - 2)
                )
                d_human = (g1.mean() - g2.mean()) / s_pool if s_pool > 0 else np.nan
            else:
                d_human = np.nan
        else:
            d_human = np.nan

        #    For twins:
        if cond_exists and len(pair) > 3:
            levels_t = pair[cond_t].unique()
            if cond_exists and len(levels_t) == 2:
                g1 = pair.loc[pair[cond_t] == levels_t[0], col_t]
                g2 = pair.loc[pair[cond_t] == levels_t[1], col_t]
                n1, n2 = len(g1), len(g2)
                s_pool = np.sqrt(
                    ((n1 - 1) * g1.var(ddof=1) + (n2 - 1) * g2.var(ddof=1)) / (n1 + n2 - 2)
                )
                d_twin = (g1.mean() - g2.mean()) / s_pool if s_pool > 0 else np.nan
            else:
                d_twin = np.nan
        else:
            d_twin = np.nan
    else:
        r = lo_r = hi_r = z_score = accuracy = mean_h = mean_t = t_stat = p_val = std_h = std_t = (
            f_stat
        ) = p_f = np.nan
        d_human = d_twin = np.nan

    results.append(
        {
            "study name": study_name,
            "persona specification": specification_name,
            "variable name": var,
            #        'variable type (raw response/DV)':     'DV',
            "correlation between the responses from humans vs. their twins": r,
            "CI_lower": lo_r,
            "CI_upper": hi_r,
            "z-score for correlation between humans vs. their twins": z_score,
            "accuracy between humans vs. their twins": accuracy,
            "mean_human": mean_h,
            "mean_twin": mean_t,
            "paired t-test t-stat": t_stat,
            "paired t-test p-value": p_val,
            "std_human": std_h,
            "std_twin": std_t,
            "variance test F-stat": f_stat,
            "variance test p-value": p_f,
            "effect size based on human": d_human,
            "effect size based on twin": d_twin,
            "domain=social?": DV_vars_social_map.get(var, np.nan),
            "domain=cognitive?": DV_vars_cognitive_map.get(var, np.nan),
            "replicating know human bias?": DV_vars_known_map.get(var, np.nan),
            "preference measure?": DV_vars_pref_map.get(var, np.nan),
            "stimuli dependent?": DV_vars_stim_map.get(var, np.nan),
            "knowledge question?": DV_vars_know_map.get(var, np.nan),
            "political question?": DV_vars_politics_map.get(var, np.nan),
            "sample size": n,
        }
    )

# results DataFrame
corr_df = pd.DataFrame(results)
print(corr_df)

# save output as csv - unit of observation is comparison between humans and twins:
out_file = f"{study_name} {specification_name} meta analysis.csv"
corr_df.to_csv(out_file, index=False)


#####participant-level data:
def make_long(df, respondent_type):
    # pick off TWIN_ID + the DVs, then melt
    long = df[["TWIN_ID"] + DV_vars].melt(
        id_vars="TWIN_ID", value_vars=DV_vars, var_name="variable_name", value_name="value"
    )

    long["respondent_type"] = respondent_type
    long["study_name"] = study_name
    long["specification_name"] = specification_name
    return long


#########################unique to this study:
# turn the index back into a column so melt can see it
df_human.reset_index(inplace=True)
df_twin.reset_index(inplace=True)
###################

# build the two halves
long_h = make_long(df_human, "human")
long_t = make_long(df_twin, "twin")

# stack them
df_long = pd.concat([long_h, long_t], ignore_index=True)

print(df_long.head())
# save output as csv - unit of observation is TWIN_ID:
out_file = f"{study_name} {specification_name} meta analysis individual level.csv"
df_long.to_csv(out_file, index=False)

print("done")

C:\Users\ot2107\AppData\Roaming\Python\Python311\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


      study name persona specification variable name  \
0  Story Beliefs       default persona    Achap1_val   
1  Story Beliefs       default persona    Achap1_aro   
2  Story Beliefs       default persona    Achap2_val   
3  Story Beliefs       default persona    Achap2_aro   
4  Story Beliefs       default persona    Bchap1_val   
5  Story Beliefs       default persona    Bchap1_aro   
6  Story Beliefs       default persona    Bchap2_val   
7  Story Beliefs       default persona    Bchap2_aro   

   correlation between the responses from humans vs. their twins  CI_lower  \
0                                           0.583898              0.495934   
1                                           0.422336              0.314989   
2                                           0.470576              0.368211   
3                                           0.248305              0.128435   
4                                           0.561564              0.470457   
5                          

C:\Users\ot2107\AppData\Local\Temp\ipykernel_344\847531666.py:162: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"{var}_min"] = min_map[var]
C:\Users\ot2107\AppData\Local\Temp\ipykernel_344\847531666.py:163: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"{var}_max"] = max_map[var]
C:\Users\ot2107\AppData\Local\Temp\ipykernel_344\847531666.py:162: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns a

In [9]:
# replicate pre-registered analysis

import numpy as np
import pandas as pd
from scipy.stats import norm, pearsonr

# 1) Re-load and process your data exactly as above
study_name = "Story Beliefs"
df_human = pd.read_csv(
    f"{study_name} human data values anonymized processed.csv", header=0
).set_index("TWIN_ID")
df_twin = pd.read_csv(
    f"{study_name} twins data values anonymized processed.csv", header=0
).set_index("TWIN_ID")


# 2) Melt function
def melt_story_data(df, source_label):
    pieces = []
    for prefix in ["A", "B"]:
        for chap in [1, 2]:
            val_col = f"{prefix}chap{chap}_val"
            aro_col = f"{prefix}chap{chap}_aro"
            if val_col in df.columns and aro_col in df.columns:
                tmp = df[[val_col, aro_col]].copy()
                tmp.columns = ["expected_valence_next", "expected_arousal_next"]
                tmp = tmp.reset_index().rename(columns={"index": "TWIN_ID"})
                tmp["story_chapter_survey"] = f"{prefix}chap{chap}"
                tmp["source"] = source_label
                pieces.append(tmp)
    return pd.concat(pieces, ignore_index=True)


# 3) Melt both
human_melted = melt_story_data(df_human, "human")
twin_melted = melt_story_data(df_twin, "twin")
combined_melted = pd.concat([human_melted, twin_melted], ignore_index=True)

# 4) Pivot wide to align human/twin for each chapter
corr_wide = combined_melted.pivot_table(
    index=["TWIN_ID", "story_chapter_survey"],
    columns="source",
    values=["expected_valence_next", "expected_arousal_next"],
).dropna()
corr_wide.columns = [f"{metric}_{src}" for metric, src in corr_wide.columns]

# 5) Compute correlation stats
n = corr_wide.shape[0]
df_stat = n - 2


def corr_stats(x, y):
    r, p = pearsonr(x, y)
    t = r * np.sqrt(df_stat / (1 - r**2))
    se_z = 1 / np.sqrt(n - 3)
    z = np.arctanh(r)
    z_crit = norm.ppf(0.975)
    lo_z, hi_z = z - z_crit * se_z, z + z_crit * se_z
    lo_r, hi_r = np.tanh(lo_z), np.tanh(hi_z)
    return r, t, df_stat, p, lo_r, hi_r


r_v, t_v, df_v, p_v, lo_v, hi_v = corr_stats(
    corr_wide["expected_valence_next_human"], corr_wide["expected_valence_next_twin"]
)
r_a, t_a, df_a, p_a, lo_a, hi_a = corr_stats(
    corr_wide["expected_arousal_next_human"], corr_wide["expected_arousal_next_twin"]
)

# 6) Display results
results = pd.DataFrame(
    {
        "Dimension": ["Valence", "Arousal"],
        "r": [r_v, r_a],
        "t": [t_v, t_a],
        "df": [df_v, df_a],
        "p-value": [p_v, p_a],
        "CI Lower": [lo_v, lo_a],
        "CI Upper": [hi_v, hi_a],
    }
)

print(results)

  Dimension         r          t    df       p-value  CI Lower  CI Upper
0   Valence  0.546342  20.648183  1002  3.356481e-79  0.501422  0.588325
1   Arousal  0.384209  13.172995  1002  1.154545e-36  0.330189  0.435721


In [10]:
n_rows = len(human_melted)
print(f"Number of rows in human_melted: {n_rows}")

n_rows = len(twin_melted)
print(f"Number of rows in twin_melted: {n_rows}")

n_rows = len(combined_melted)
print(f"Number of rows in combined_melted: {n_rows}")

Number of rows in human_melted: 1004
Number of rows in twin_melted: 1004
Number of rows in combined_melted: 2008
